In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator
)
import torch
import numpy as np
from sklearn.model_selection import train_test_split

import os

os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

PRETRAINED = "../../shared/models/ru-e5-large"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
model = AutoModelForQuestionAnswering.from_pretrained(PRETRAINED).to(device)

Using device: cuda


Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at ../../shared/models/ru-e5-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# import jax
# print(jax.__file__)

In [3]:
# !pip show -f jax

In [4]:
# Load your CSV
df = pd.read_csv("extractive_summ_data.csv")

# Convert DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train and validation
train_val = dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = train_val["train"]
val_dataset = train_val["test"]

# Create DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [5]:
train_dataset[0]

{'Unnamed: 0': 2847,
 'id': '243c1e75-9e07-4893-a977-80e9c7457ac5',
 'context': 'X5 Retail опубликовала отчетность без особых сюрпризов, рынок отреагировал сдержанно. Тем не менее, аналитики продолжают следить за ситуацией, особенно в контексте валютных колебаний и изменения сырьевых цен.\nНа фоне ожиданий по ставке ЦБ, многие участники рынка заняли выжидательную позицию.\nГазпром опубликовала отчетность без особых сюрпризов, рынок отреагировал сдержанно. Тем не менее, аналитики продолжают следить за ситуацией, особенно в контексте валютных колебаний и изменения сырьевых цен.\nРубль стабилизировался после волатильной недели, но долгосрочные перспективы остаются неясными.\nOzon в последнее время демонстрирует неоднозначную динамику. С одной стороны, финансовые показатели за квартал превзошли ожидания аналитиков, с другой — сохраняется высокая волатильность на фоне глобальных рисков. Пока инвесторы сохраняют осторожность, несмотря на краткосрочные сигналы роста.\nTotalEnergies опубликова

In [9]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answer', 'answer_start', 'answer_end', '__index_level_0__'],
        num_rows: 14517
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answer', 'answer_start', 'answer_end', '__index_level_0__'],
        num_rows: 2562
    })
})

In [10]:
def prepare_features(examples):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True
    )

    # Save a copy of offset mapping for later (metrics)
    offset_mappings = tokenized["offset_mapping"]

    # Compute start/end token positions
    start_positions = []
    end_positions = []

    for i in range(len(examples["answer"])):
        offsets = offset_mappings[i]
        sequence_ids = tokenized.sequence_ids(i)
        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        answer_start = examples["answer_start"][i]
        answer_end = examples["answer_end"][i]

        # Find context token span
        context_start = next(i for i, sid in enumerate(sequence_ids) if sid == 1)
        context_end = max(i for i, sid in enumerate(sequence_ids) if sid == 1)

        if not (answer_start >= offsets[context_start][0] and answer_end <= offsets[context_end][1]):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start = context_start
            while start < len(offsets) and offsets[start][0] <= answer_start:
                start += 1
            end = context_end
            while end >= 0 and offsets[end][1] >= answer_end:
                end -= 1
            start_positions.append(start - 1)
            end_positions.append(end + 1)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions

    # Keep offset_mapping only for eval
    tokenized["offset_mapping"] = offset_mappings
    tokenized["example_id"] = examples["id"]
    return tokenized


# Store offset mappings for validation set separately
tokenized_datasets = dataset_dict.map(
    prepare_features,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
    desc="Tokenizing datasets"
)


# Save tokenized validation data
tokenized_validation = dataset_dict["validation"].map(
    prepare_features,
    batched=True,
    remove_columns=dataset_dict["validation"].column_names
)
validation_features = tokenized_validation

offset_mappings = validation_features["offset_mapping"]
example_ids = validation_features["example_id"]

Tokenizing datasets:   0%|          | 0/14517 [00:00<?, ? examples/s]

Tokenizing datasets:   0%|          | 0/2562 [00:00<?, ? examples/s]

Map:   0%|          | 0/2562 [00:00<?, ? examples/s]

In [11]:
def show_predictions(index):
    encoded = tokenized_datasets["validation"][index]
    decoded = tokenizer.decode(encoded["input_ids"], skip_special_tokens=False)

    print("Encoded input:", decoded)

    tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"])
    print("Tokens:", tokens)

    start_pos = encoded["start_positions"]
    end_pos = encoded["end_positions"]
    print(f"Answer span: {tokens[start_pos]} ... {tokens[end_pos]}")
    print(f"Answer text: {' '.join(tokens[start_pos:end_pos+1])}")

In [12]:
# show_predictions(0)

In [13]:
import numpy as np
import evaluate
from collections import defaultdict

squad_metric = evaluate.load("squad")

from transformers import EvalPrediction
cached_features = tokenized_datasets["validation"]
cached_raw_dataset = val_dataset


def compute_metrics(eval_prediction):
    start_logits, end_logits = eval_prediction.predictions
    predictions = []
    references = []

    for i, feature in enumerate(cached_features):
        offset_mapping = feature["offset_mapping"]
        context = cached_raw_dataset[i]["context"]
        answer = cached_raw_dataset[i]["answer"]
        answer_start = cached_raw_dataset[i]["answer_start"]

        start_idx = int(np.argmax(start_logits[i]))
        end_idx = int(np.argmax(end_logits[i]))

        if (
            start_idx >= len(offset_mapping)
            or end_idx >= len(offset_mapping)
            or offset_mapping[start_idx] is None
            or offset_mapping[end_idx] is None
            or end_idx < start_idx
        ):
            pred_text = ""
        else:
            start_char = offset_mapping[start_idx][0]
            end_char = offset_mapping[end_idx][1]
            pred_text = context[start_char:end_char]

        predictions.append({"id": feature["example_id"], "prediction_text": pred_text})
        references.append({
            "id": feature["example_id"],
            "answers": {
                "text": [answer],
                "answer_start": [answer_start]
            }
        })

    return squad_metric.compute(predictions=predictions, references=references)


In [14]:
from functools import partial

compute_metrics_with_data = partial(
    compute_metrics,
    features=validation_features,
    raw_dataset=val_dataset
)


In [15]:
tokenized_datasets["validation"]

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
    num_rows: 2562
})

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    # save_strategy="steps",
    # load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    logging_strategy="epoch",
    eval_steps=20,
    eval_accumulation_steps=5
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1275719/1307781823.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Exact Match,F1
20,No log,1.958750,12.841530,43.877486
40,No log,0.732234,63.817330,74.692714
60,No log,0.247621,93.754879,96.297110
80,No log,0.099337,95.979703,97.106317
100,No log,0.048475,98.360656,99.268762
120,No log,0.034709,98.946136,99.461170
140,No log,0.015240,99.141296,99.687807
160,No log,0.011656,99.648712,99.816743
180,No log,0.011971,99.570648,99.748795
200,No log,0.006470,99.609680,99.925223


KeyboardInterrupt: 

In [18]:
model.save_pretrained("./qa_model_russian")
tokenizer.save_pretrained("./qa_model_russian")

('./qa_model_russian/tokenizer_config.json',
 './qa_model_russian/special_tokens_map.json',
 './qa_model_russian/sentencepiece.bpe.model',
 './qa_model_russian/added_tokens.json',
 './qa_model_russian/tokenizer.json')

### Test

In [55]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "./qa_model_russian"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path).to(device)

# Question
question = "Яндекс"

# Initial context text
context = """
В Госдепе США сообщили, что Вашингтон отказывается от роли посредника в диалоге между Россией и Украиной, но будет продолжать работать над достижением мира, однако разрешение ситуации зависит от «конкретных действий» со стороны Москвы и Киева.  

Не добавляет и позитива динамика цены на нефть. Попытки развить вчерашний отскок не реализовались, и котировки опустились ниже $61 за баррель.

Акции газовых компаний — Газпрома и НОВАТЭКа — сегодня среди аутсайдеров. 

Газпром представил в среду сильные финансовые результаты за 2024 г. Расчетный дивиденд 30,3 руб. Главный вопрос — вернется ли корпорация к распределению прибыли впервые с 2022 г. Выплаты не заложены в бюджет, что добавляет неопределенности вкупе к геополитической повестке в целом.  

Бумаги НОВАТЭКа несут потери четвертый день подряд. Ближайшая поддержка — 200-дневная скользящая средняя на 1121 руб. Кроме общерыночного пессимизма, давление оказывает и снижение цен на нефть. Финансовые показатели компании зависят, в частности, от цен на черное золото (компания продает продукты переработки газового конденсата, а долгосрочные СПГ-контракты привязаны к котировкам Brent).

Бумаги Полюса пытаются отскочить после семи дней падения. Цены на золото в пятницу вышли в умеренный плюс после коррекционного снижения от исторических максимумов. Также поддержку оказывает и резкое снижение рубля. Для выхода на уверенную траекторию роста нужна поддержка со стороны цен на базовый актив.

Акции ВТБ во второй половине дня сократили дневные потери. Поддержку бумагам оказывает дивидендный фактор. Набсовет банка рекомендовал выплатить дивиденды за 2024 г. в размере 25,58 руб. на одну акцию. Текущая дивидендная доходность — около 26,3%. «Дивидендный сюрприз» стал драйвером роста бумаг в понедельник. Сдерживает оптимизм вероятность проведения допэмиссии для пополнения капитала, но ее параметры пока неизвестны.

Префы Транснефти смотрятся сильнее коллег по сектору. Здесь тоже в фокусе дивиденды за 2024 г. После выхода отчета за прошлый год, мы оцениваем их размер в 190 руб. на акцию (дивидендная доходность 15%). Кроме того, доходы компании не зависят от ценовой конъюнктуры на рынке нефти, где баррель Brent за месяц подешевел на 15%.

Обычка БСП — в небольшом минусе и смотрится лучше большинства представителей Индекса МосБиржи. Акции на текущей неделе переписали свои исторические максимумы — теперь это 416,97 руб., но уже в понедельник ждем серьезного падения. Причина — бумаги 5 мая очистятся от финальных дивидендов за 2024 г.  Выплаты составят 29,72 руб. на обычку, что предполагает текущую дивидендную доходность на уровне 7,3%. С учетом налогов на старте следующе неделе они могут потерять 6,4%.

Акции Nanduq (бывшая Qiwi) сегодня подскочили на 21%. Поводом для резкого скачка вверх могло стать уведомление ликвидатора Киви-Банка акционерам о наличии и праве акционеров на получение имущества, оставшегося после завершения расчетов с кредиторами.
"""

context = """
Главное
• Индекс МосБиржи вырос почти на 3% во вторник после пятидневного снижения.

• Цены на нефть вчера провоцировали продажи, а сегодня поддерживают покупателей, прибавляя 4%.

• Акции Газпрома в фокусе.

• Лидеры: Россети Северо-Запад (+7,5%), БСП-ап (+3,8%), ПИК (+5,2%), Русснефть (+5%).

• Аутсайдеры: NanduQ (-1,4%), Башнефть-ао (-0,8%), Россети Урал (-0,3%), ВИ.ру (-0,2%).

В деталях
Рынок акций сегодня восстанавливается после 10%-ого снижения от максимумов апреля. Внушительный отскок, который перекрывает всю просадку понедельника выглядит позитивно, хотя до поддержки растущего тренда последних месяцев в районе 2650 индекс не дошел. 

При достаточно активном движении обороты лишь приближаются к средним — сказывается торговый период между длинными выходными. В составе индекса наторговали на 82 млрд руб. 

Никуда не исчезли и геополитические причины сдерживающие активность. Агентство Reuters сообщило, что ЕС предлагает включить в 17-й пакет санкций против России более 100 судов, связанных с российским «теневым флотом», и еще 60 физлиц и юрлиц.

Внешний фон поддержал отскок цен на нефть. Котировки Brent вчера опускались ниже $59 за баррель, а сегодня вернулись к $62,5. Здесь могли сказаться вышеназванные угрозы ЕС против РФ, возвращение к торгам трейдеров крупнейшего импортера нефти —Китая после длинных выходных, банальный спекулятивный отскок.

На прошлой неделе в фокусе инвесторов были акции Газпрома ввиду ожидавшегося отчета, который оказался неплохим, хотя аналитики сдержанно оценивают перспективы высоких дивидендов. Сегодня бумага остается на первой строчке по оборотам на фоне ряда новостей.

Во-первых, в начале вечерней сессии глава Минфина Антон Силуанов подтвердил, что в бюджет 2025 г. не заложены дивиденды Газпрома. Но также он отметил, что задействовать специальные меры, в том числе налоговые, для повышения доходов бюджета в этом году не планируется. Откат на этих новостях был скоротечным. 

Во вторых, Еврокомиссия представила «дорожную карту» отказа от российских энергоресурсов, но и это не сильно расстроило инвесторов, с учетом того, что ЕС еще в 2022 г. озвучил подобные планы. В то же время, помощник президента РФ Юрий Ушаков заявил, что президент РФ Владимир Путин и председатель КНР Си Цзиньпин 8 мая обсудят широкий круг вопросов, включая конфликт на Украине, энергетику. Относительно последней инвесторов интересует, конечно, вопрос «Силы Сибири-2». Акции Газпрома выросли на 2,5% на основных торгах.

Среди лидеров сегодня были акции ПИК, Самолета, ВК, Системы, то есть компаний чувствительных к высоким процентным ставкам, которые и падали с опережением. Отдельно здесь стоит сказать о ВК, акционеры которой одобрили допэмиссию. ВК таким образом планирует снизить долговую нагрузку. Размер выпуска — до 115 млрд руб. Цена акции при допэмиссии — 324,9 руб., на 33% выше текущей. Аналитики БКС учли в своей модели допэмиссию и сохраняют нейтральный взгляд на бумагу с целевой ценой 300 руб.

Несмотря на отскок цен на нефть, более сильную динамику сегодня показали не Роснефть (+1,9%), ЛУКОЙЛ (+1,4%) и Татнефть (+1,3%), а Яндекс (+4,7%), Сбер (+2,8%) и Т-Технологии (+3,1%), то есть большая часть краткосрочных фаворитов аналитиков БКС.

Обоснованный рост продолжается в акциях Полюса. Котировки золота смогли вчера отскочить от минимумов с середины апреля и уже превысили отметку $3400. 

Несколько настораживает, что сегодняшний оптимизм на рынке акций не был поддержан столько же активной динамикой ОФЗ. RGBI прибавил лишь 0,06%. Завтра Минфин проведет аукцион по размещению облигаций 26233, а Росстат представит данные по недельной инфляции.

Рубль пока по-прежнему не спешит снижаться. Официальный курс доллара опустился ниже 81.
"""

# Split the context by newlines into paragraphs
paragraphs = context.strip().split("\n")

# Run QA on each paragraph and select the best
best_answer = ""
best_score = float('-inf')

for paragraph in paragraphs:
    inputs = tokenizer(question, paragraph, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores) + 1

    answer_tokens = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    score = start_scores[0][start_idx] + end_scores[0][end_idx - 1]
    if score > best_score:
        best_score = score
        best_answer = answer

print(f"Best Answer: {best_answer}")


Best Answer: Яндекс (+4,7%), Сбер (+2,8%) и Т-Технологии (+3,1%), то есть большая часть краткосрочных фаворитов аналитиков БКС.
